In [1]:
from scripts import cwl_correction_raw
import matplotlib.pyplot as plt
import numpy as np
import mne
import os
import pandas as pd
from scipy import signal
correction = 'ref' 
rootDir = '/Users/moyne/Documents/GitHub/neuxus_test/Post_corrections/Reference'
rootDir2 = '/Vision/eeg/exp/GA_PA_CWL'
rootDir22 = '/Vision/eeg/exp/GA_CWL'
rootDir3 = '/Users/moyne/Documents/GitHub/neuxus_test/RDA/GA_PA_Corrections_RecView_replay'
rootDir4 = '/Users/moyne/Documents/GitHub/neuxus_test/RDA/GA_Corrections_RecView_replay'
rootDir5 = '/Users/moyne/Documents/GitHub/neuxus_test/Post_corrections/Neuxus/GA'
rootDir6 = '/Users/moyne/Documents/GitHub/neuxus_test/Post_corrections/Neuxus/GA_CWL'
rootDir7 = '/Users/moyne/Documents/GitHub/neuxus_test/Post_corrections/Neuxus/GA_PA'

SubList = ['02','03','04','05','06','07','08','09','10','11','12']

electrodes_OI_right = ['P4', 'P8', 'O2']
electrodes_OI_left = ['P3', 'P7', 'O1']
bands = [([8, 13], 'alpha'), ([4, 8], 'theta'), ([1, 4], 'delta'), ([13, 15], 'beta1'), ([15, 25], 'beta2')]

dicts = list()
for subject in SubList:
    d = {}
    d['subject'] = subject
    for condition in ['eyes_closed', 'eyes_open']:
        file_path = os.path.join(rootDir, f"P{subject:02}_{condition}-ref-annotated-raw.fif") # before ica, after filtering, notch, rereferencing, downsampling 
        ref_raw = mne.io.read_raw(file_path, preload=True)
        ref_raw.set_montage('standard_1020', on_missing='warn')
        ref_raw.set_channel_types({'CWL1': 'misc', 'CWL2': 'misc', 'CWL3': 'misc', 'CWL4': 'misc', 'ECG': 'ecg'})
        # Apply CWL correction
        raw_corrected = cwl_correction_raw(ref_raw, eeg_picks=mne.pick_types(ref_raw.info, eeg=True), cwl_picks=mne.pick_types(ref_raw.info, misc=True), time_delay=21e-3, window_duration=4, overlap=0.5)
        
        
        file_path_GA_PA_CWL_Analyzer = os.path.join(rootDir2, f"Analyzer_GA_PA_CWL_P{subject:02}_{condition}_mrion-edf.edf")#f"P{subject:02}_{condition}_mrion_CWL Regression.vhdr
        GA_PA_CWL_Analyzer = mne.io.read_raw_edf(file_path_GA_PA_CWL_Analyzer, preload=True)
        GA_PA_CWL_Analyzer.set_montage('standard_1020', on_missing='warn')
        GA_PA_CWL_Analyzer.set_channel_types({'CWL1': 'misc', 'CWL2': 'misc', 'CWL3': 'misc', 'CWL4': 'misc', 'ECG': 'ecg'})

        file_path_GA_CWL_Analyzer = os.path.join(rootDir22, f"Analyzer_GA_CWL_P{subject:02}_{condition}_mrion-edf.edf")#f"P{subject:02}_{condition}_mrion_CWL Regression.vhdr
        GA_CWL_Analyzer = mne.io.read_raw_edf(file_path_GA_CWL_Analyzer, preload=True)
        GA_CWL_Analyzer.set_montage('standard_1020', on_missing='warn')
        GA_CWL_Analyzer.set_channel_types({'CWL1': 'misc', 'CWL2': 'misc', 'CWL3': 'misc', 'CWL4': 'misc', 'ECG': 'ecg'})


        file_path_GA_PA_RecView = os.path.join(rootDir3, f"P{subject:02}_{condition}_mrion_RecView_PA_GA_raw.fif") # before ica, after filtering, notch, rereferencing, downsampling 
        GA_PA_RecView = mne.io.read_raw(file_path_GA_PA_RecView, preload=True)
        GA_PA_RecView.set_montage('standard_1020', on_missing='warn')
        mne.rename_channels(GA_PA_RecView.info,{'1':'Fp1','2':'Fp2','3':'F3','4':'F4','5':'C3','6':'C4','7':'P3','8':'P4','9':'O1','10':'O2','11':'F7',\
			'12':'F8','13':'T7','14':'T8','15':'P7','16':'P8','17':'Fz','18':'Cz','19':'Pz','20':'Oz',\
				'21':'FC1','22':'FC2','23':'CP1','24':'CP2','25':'FC5','26':'FC6','27':'CP5','28':'CP6','29':'TP9','30':'TP10','31':'POz','32':'ecg',
                '33':'CWL1','34':'CWL2','35':'CWL3','36':'CWL4'})
        GA_PA_RecView.ch_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg',
           'eeg', 'eeg', 'eeg', 'eeg','eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 
           'eeg', 'eeg', 'ecg','misc','misc','misc','misc']
        #GA_PA_RecView.set_channel_types({'CWL1': 'misc', 'CWL2': 'misc', 'CWL3': 'misc', 'CWL4': 'misc', 'ECG': 'ecg'})
        Fs = GA_PA_RecView.info['sfreq']
        info = mne.create_info(ch_names=GA_PA_RecView.ch_names,sfreq=Fs,ch_types=GA_PA_RecView.ch_types)
        GA_PA_RecView.info = info
        montage = mne.channels.make_standard_montage('standard_1020') 
        GA_PA_RecView.set_montage(montage,match_case=False)
        # Remove inf data
        data = GA_PA_RecView.get_data() 
        data = np.nan_to_num(data, copy=False, nan=0.0, neginf=data[data != -np.inf].max(), posinf=data[data != np.inf].max()) 
        GA_PA_RecView._data = data
        # Apply CWL correction
        GA_PA_CWL_RecView = cwl_correction_raw(GA_PA_RecView,
                                               eeg_picks=mne.pick_types(GA_PA_RecView.info, eeg=True),
                                               cwl_picks=mne.pick_types(GA_PA_RecView.info, misc=True),
                                               time_delay=21e-3, window_duration=4, overlap=0.5)
        
        
        file_path_GA_RecView = os.path.join(rootDir4, f"P{subject:02}_{condition}_mrion_RecView_GA_raw.fif") # before ica, after filtering, notch, rereferencing, downsampling 
        GA_RecView = mne.io.read_raw(file_path_GA_RecView, preload=True)
        GA_RecView.set_montage('standard_1020', on_missing='warn')
        mne.rename_channels(GA_RecView.info,{'1':'Fp1','2':'Fp2','3':'F3','4':'F4','5':'C3','6':'C4','7':'P3','8':'P4','9':'O1','10':'O2','11':'F7',\
			'12':'F8','13':'T7','14':'T8','15':'P7','16':'P8','17':'Fz','18':'Cz','19':'Pz','20':'Oz',\
				'21':'FC1','22':'FC2','23':'CP1','24':'CP2','25':'FC5','26':'FC6','27':'CP5','28':'CP6','29':'TP9','30':'TP10','31':'POz','32':'ecg',
                '33':'CWL1','34':'CWL2','35':'CWL3','36':'CWL4'})
        GA_RecView.ch_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg',
           'eeg', 'eeg', 'eeg', 'eeg','eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 
           'eeg', 'eeg', 'ecg','misc','misc','misc','misc']
        #GA_PA_RecView.set_channel_types({'CWL1': 'misc', 'CWL2': 'misc', 'CWL3': 'misc', 'CWL4': 'misc', 'ECG': 'ecg'})
        Fs = GA_RecView.info['sfreq']
        info = mne.create_info(ch_names=GA_RecView.ch_names,sfreq=Fs,ch_types=GA_RecView.ch_types)
        GA_RecView.info = info
        montage = mne.channels.make_standard_montage('standard_1020') 
        GA_RecView.set_montage(montage,match_case=False)
        # Remove inf data
        data = GA_RecView.get_data() 
        data = np.nan_to_num(data, copy=False, nan=0.0, neginf=data[data != -np.inf].max(), posinf=data[data != np.inf].max()) 
        GA_RecView._data = data
        data=data().resample(sfreq=250)
        # Apply CWL correction
        GA_CWL_RecView = cwl_correction_raw(GA_RecView,
                                               eeg_picks=mne.pick_types(GA_RecView.info, eeg=True),
                                               cwl_picks=mne.pick_types(GA_RecView.info, misc=True),
                                               time_delay=21e-3, window_duration=4, overlap=0.5)

        
        file_path_GA_Neuxus = os.path.join(rootDir5, f"Neuxus_GA_P{subject:02}_{condition}_mrion_raw.fif")#f"P{subject:02}_{condition}_mrion_CWL Regression.vhdr
        GA_Neuxus = mne.io.read_raw(file_path_GA_Neuxus, preload=True)
        GA_Neuxus.set_montage('standard_1020', on_missing='warn')
        GA_Neuxus.set_channel_types({'CWL1': 'misc', 'CWL2': 'misc', 'CWL3': 'misc', 'CWL4': 'misc', 'ECG': 'ecg'})

        file_path_GA_CWL_Neuxus = os.path.join(rootDir6, f"Neuxus_GA_CWL_P{subject:02}_{condition}_mrion_raw.fif")#f"P{subject:02}_{condition}_mrion_CWL Regression.vhdr
        GA_CWL_Neuxus = mne.io.read_raw(file_path_GA_CWL_Neuxus, preload=True)
        GA_CWL_Neuxus.set_montage('standard_1020', on_missing='warn')
        GA_CWL_Neuxus.set_channel_types({'CWL1': 'misc', 'CWL2': 'misc', 'CWL3': 'misc', 'CWL4': 'misc', 'ECG': 'ecg'})

        file_path_GA_PA_Neuxus = os.path.join(rootDir7, f"Neuxus_GA_PA_P{subject:02}_{condition}_mrion_raw.fif")#f"P{subject:02}_{condition}_mrion_CWL Regression.vhdr
        GA_PA_Neuxus = mne.io.read_raw(file_path_GA_PA_Neuxus, preload=True)
        GA_PA_Neuxus.set_montage('standard_1020', on_missing='warn')
        GA_PA_Neuxus.set_channel_types({'CWL1': 'misc', 'CWL2': 'misc', 'CWL3': 'misc', 'CWL4': 'misc', 'ECG': 'ecg'})

        


        
        # Plot PSD comparisons
        eeg_channels = ref_raw.pick_types(eeg=True).ch_names
        for channel in eeg_channels:
            pre = ref_raw.get_data(picks=[channel])[0]
            post = raw_corrected.get_data(picks=[channel])[0]

            c = np.corrcoef(pre, post)[0,1]
            d[f'{condition}_correlation_prepost_{channel}'] = c

            data = np.array([pre, post])
            # Pre : Fp1, Post: Oz
            info = mne.create_info(ch_names=['Fp1', 'Oz'], sfreq=ref_raw.info['sfreq'], ch_types=['eeg', 'eeg'])
            raw_comparison = mne.io.RawArray(data=data, info=info)
            raw_comparison.set_montage('standard_1020')
            raw_comparison.save(f'Post_corrections/Reference/CWL_python_block/P{subject:02}_{condition}-comparison_rawVSrawCWL-{channel}-raw.fif', overwrite=True)

            for freq in [6, 20]:
                plt.figure()
                raw_comparison.plot_psd(fmax=freq, spatial_colors=True, average=False, show=False)
                plt.savefig(f'Post_corrections/Reference/CWL_python_block/P{subject:02}_{condition}-{correction}-{channel}-{freq}.png', dpi=300)
            
    
        for raw, raw_name in [(ref_raw, 'reference'), (raw_corrected, 'corrected'), (GA_PA_CWL_Analyzer, 'GA_PA_CWL_Analyzer'), (GA_CWL_Analyzer, 'GA_CWL_Analyzer'), (GA_PA_RecView, 'GA_PA_RecView'),(GA_PA_CWL_RecView, 'GA_PA_CWL_RecView'), (GA_RecView, 'GA_RecView'),(GA_CWL_RecView, 'GA_CWL_RecView'), (GA_Neuxus, 'GA_Neuxus'),(GA_CWL_Neuxus, 'GA_CWL_Neuxus'), (GA_PA_Neuxus, 'GA_PA_Neuxus')]: # (ref_raw_my_new_condition, 'my_new_condition')
            for freqs, band_name in bands:
                # Right
                tf = mne.time_frequency.psd_array_welch(raw.get_data( picks=electrodes_OI_right), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
                power_right_raw = np.average(np.average(tf[0], axis=0), axis=0)
                # Left
                tf = mne.time_frequency.psd_array_welch(raw.get_data( picks=electrodes_OI_left), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
                power_left_raw = np.average(np.average(tf[0], axis=0), axis=0)
                # Ratio
                ratio_raw =((power_left_raw-power_right_raw)/power_right_raw)*100
                # Mean
                mean_raw = (power_right_raw + power_left_raw)/2

                d[f'{condition}_{raw_name}_{band_name}_power_right'] = power_right_raw
                d[f'{condition}_{raw_name}_{band_name}_power_left'] = power_left_raw
                d[f'{condition}_{raw_name}_{band_name}_power_ratio'] = ratio_raw
                d[f'{condition}_{raw_name}_{band_name}_power_mean'] = mean_raw

    dicts.append(d)

df = pd.DataFrame(dicts)
df.to_csv(f'Post_corrections/Reference/CWL_python_block/data.csv')


Opening raw data file /Users/moyne/Documents/GitHub/neuxus_test/Post_corrections/Reference\P02_eyes_closed-ref-annotated-raw.fif...
    Range : 7500 ... 52499 =     30.000 ...   209.996 secs
Ready.
Reading 0 ... 44999  =      0.000 ...   179.996 secs...


c:\Users\moyne\Documents\GitHub\neuxus_test\scripts\cwl.py:41: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coeffs = np.linalg.lstsq(cwl_segment.T, eeg_segment.T, )[0]


Creating RawArray with float64 data, n_channels=31, n_times=45000
    Range : 0 ... 44999 =      0.000 ...   179.996 secs
Ready.
Extracting EDF parameters from c:\Vision\eeg\exp\GA_PA_CWL\Analyzer_GA_PA_CWL_P02_eyes_closed_mrion-edf.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 66499  =      0.000 ...   265.996 secs...
Extracting EDF parameters from c:\Vision\eeg\exp\GA_CWL\Analyzer_GA_CWL_P02_eyes_closed_mrion-edf.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 66499  =      0.000 ...   265.996 secs...
Opening raw data file /Users/moyne/Documents/GitHub/neuxus_test/RDA/GA_PA_Corrections_RecView_replay\P02_eyes_closed_mrion_RecView_PA_GA_raw.fif...
Isotrak not found
    Range : 0 ... 64284 =      0.000 ...   257.136 secs
Ready.
Reading 0 ... 64284  =      0.000 ...   257.136 secs...


C:\Users\moyne\AppData\Local\Temp\ipykernel_28588\1349239113.py:39: RuntimeWarning: DigMontage is only a subset of info. There are 5 channel positions not present in the DigMontage. The channels missing from the montage are:

['ECG', 'CWL1', 'CWL2', 'CWL3', 'CWL4'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.
  GA_PA_CWL_Analyzer.set_montage('standard_1020', on_missing='warn')
C:\Users\moyne\AppData\Local\Temp\ipykernel_28588\1349239113.py:40: RuntimeWarning: The unit for channel(s) CWL1, CWL2, CWL3, CWL4 has changed from V to NA.
  GA_PA_CWL_Analyzer.set_channel_types({'CWL1': 'misc', 'CWL2': 'misc', 'CWL3': 'misc', 'CWL4': 'misc', 'ECG': 'ecg'})
C:\Users\moyne\AppData\Local\Temp\ipykernel_28588\1349239113.py:44: RuntimeWarning: DigMontage is only a subset of info. There are 5 channel positions not pre

Creating RawArray with float64 data, n_channels=31, n_times=64285
    Range : 0 ... 64284 =      0.000 ...   257.136 secs
Ready.
Opening raw data file /Users/moyne/Documents/GitHub/neuxus_test/RDA/GA_Corrections_RecView_replay\P02_eyes_closed_mrion_RecView_GA_raw.fif...
Isotrak not found
    Range : 0 ... 62988 =      0.000 ...   251.952 secs
Ready.
Reading 0 ... 62988  =      0.000 ...   251.952 secs...


C:\Users\moyne\AppData\Local\Temp\ipykernel_28588\1349239113.py:77: RuntimeWarning: DigMontage is only a subset of info. There are 36 channel positions not present in the DigMontage. The channels missing from the montage are:

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.
  GA_RecView.set_montage('standard_1020', on_missing='warn')


TypeError: 'numpy.ndarray' object is not callable

In [ ]:
test_raw.ch_names

In [ ]:
test_raw = GA_PA_RecView.copy()

data = test_raw.get_data() 
data = np.nan_to_num(data, copy=False, nan=0.0, neginf=data[data != -np.inf].max(), posinf=data[data != np.inf].max()) 
test_raw._data = data

GA_PA_CWL_RecView = cwl_correction_raw(test_raw, eeg_picks=mne.pick_types(test_raw.info, eeg=True), cwl_picks=mne.pick_types(test_raw.info, misc=True), time_delay=21e-3, window_duration=4, overlap=0.5)


In [ ]:
data.min()

In [ ]:
data = GA_PA_RecView.get_data() 
data = np.nan_to_num(data, copy=False, nan=0.0, neginf=data[data != -np.inf].max(), posinf=data[data != np.inf].max()) 
data

In [ ]:
# theta
# Define the electrodes of interest
electrodes_OI_right_theta = ['F4', 'AF4', 'FC2']
electrodes_OI_left_theta = ['F3', 'AF3', 'FC1']
electrodes_OI_theta =['Fz', 'FC2', 'FC1', 'F3', 'F4']
freqs = [5, 8]


tf = mne.time_frequency.psd_array_welch(raw.get_data( picks=electrodes_OI_right_theta), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
theta_power_right_raw = np.average(np.average(tf[0], axis=0), axis=0)

tf = mne.time_frequency.psd_array_welch(raw.get_data( picks=electrodes_OI_left_theta), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
theta_power_left_raw = np.average(np.average(tf[0], axis=0), axis=0)

theta_ratio_raw =((theta_power_left_raw-theta_power_right_raw)/theta_power_right_raw)*100

theta_mean_raw=(theta_power_right_raw+theta_power_left_raw)/2

tf = mne.time_frequency.psd_array_welch(raw.get_data( picks=electrodes_OI_theta), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
theta_power_midline_raw = np.average(np.average(tf[0], axis=0), axis=0)

tf = mne.time_frequency.psd_array_welch(raw_corrected.get_data( picks=electrodes_OI_right_theta), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
theta_power_right_corrected = np.average(np.average(tf[0], axis=0), axis=0)

tf = mne.time_frequency.psd_array_welch(raw_corrected.get_data( picks=electrodes_OI_left_theta), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
theta_power_left_corrected = np.average(np.average(tf[0], axis=0), axis=0)

theta_ratio_corrected=((theta_power_left_corrected-theta_power_right_corrected)/theta_power_right_corrected)*100

theta_mean_corrected=(theta_power_right_corrected+theta_power_left_corrected)/2

tf = mne.time_frequency.psd_array_welch(raw_corrected.get_data( picks=electrodes_OI_theta), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
theta_power_midline_corrected = np.average(np.average(tf[0], axis=0), axis=0)

theta_ratio1=alpha_ratio_raw-theta_ratio_corrected

theta_ratio2=(theta_power_left_raw-theta_power_left_corrected)-(theta_power_right_raw-theta_power_right_corrected)/(theta_power_left_raw-theta_power_left_corrected)+(theta_power_right_raw-theta_power_right_corrected)

cFz=np.corrcoef(preFz, postFz)
cFz=cFz[0,1]

print(cFz)

Variable_theta=[[cFz,theta_power_left_raw, theta_power_right_raw, theta_ratio_raw, theta_mean_raw, theta_power_midline_raw, theta_power_left_corrected, theta_power_right_corrected, theta_ratio_corrected, theta_mean_corrected, theta_power_midline_corrected, theta_ratio1, theta_ratio2]]
#Headers_theta=["pre-post Fz coef corr","theta_power_left_raw", "theta_power_right_raw", "theta_ratio_raw", "theta_mean_raw", "theta_midline_raw", "theta_power_left_corrected", "theta_power_right_corrected", "theta_ratio_corrected", "theta_mean_corrected","theta_ratio1", "theta_ratio2"]
#row_name= subject
DF = pd.DataFrame(Variable_theta) 
#DF.rename(columns={"pre-post Fz coef corr": "0", "theta_power_left_raw": "1","theta_power_right_raw":"2", "theta_ratio_raw":"3", "theta_mean_raw":"4", "theta_midline_raw":"5", "theta_power_left_corrected":"6", "theta_power_right_corrected":"7","theta_ratio_corrected":"8", "theta_mean_corrected":"9", "theta_power_midline_corrected":"10", "theta_ratio1":"11", "theta_ratio2":"12"})
DF.rename(columns={"pre-post Fz coef corr": "11", "theta_power_left_raw": "12","theta_power_right_raw":"13", "theta_ratio_raw":"14", "theta_mean_raw":"15", "theta_midline_raw":"16", "theta_power_left_corrected":"17", "theta_power_right_corrected":"18","theta_ratio_corrected":"19", "theta_mean_corrected":"20", "theta_power_midline_corrected":"21", "theta_ratio1":"22", "theta_ratio2":"23"})


In [ ]:
# beta1: low beta (sensorimotor rhythm - SMR)
# Define the electrodes of interest
electrodes_OI_right_beta1 = ['C4']
electrodes_OI_left_beta1 = ['C3']
electrodes_OI_beta1 =['Cz', 'C3', 'C4']
freqs = [12, 15]


tf = mne.time_frequency.psd_array_welch(raw.get_data( picks=electrodes_OI_right_beta1), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
beta1_power_right_raw = np.average(np.average(tf[0], axis=0), axis=0)

tf = mne.time_frequency.psd_array_welch(raw.get_data( picks=electrodes_OI_left_beta1), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
beta1_power_left_raw = np.average(np.average(tf[0], axis=0), axis=0)

beta1_ratio_raw =((beta1_power_left_raw-beta1_power_right_raw)/beta1_power_right_raw)*100

beta1_mean_raw=(beta1_power_right_raw+beta1_power_left_raw)/2

tf = mne.time_frequency.psd_array_welch(raw.get_data( picks=electrodes_OI_beta1), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
beta1_power_midline_raw = np.average(np.average(tf[0], axis=0), axis=0)

tf = mne.time_frequency.psd_array_welch(raw_corrected.get_data( picks=electrodes_OI_right_beta1), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
beta1_power_right_corrected = np.average(np.average(tf[0], axis=0), axis=0)

tf = mne.time_frequency.psd_array_welch(raw_corrected.get_data( picks=electrodes_OI_left_beta1), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
beta1_power_left_corrected = np.average(np.average(tf[0], axis=0), axis=0)

beta1_ratio_corrected=((beta1_power_left_corrected-beta1_power_right_corrected)/beta1_power_right_corrected)*100

beta1_mean_corrected=(beta1_power_right_corrected+beta1_power_left_corrected)/2

tf = mne.time_frequency.psd_array_welch(raw_corrected.get_data( picks=electrodes_OI_beta1), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
beta1_power_midline_corrected = np.average(np.average(tf[0], axis=0), axis=0)

beta1_ratio1=alpha_ratio_raw-beta1_ratio_corrected

beta1_ratio2=(beta1_power_left_raw-beta1_power_left_corrected)-(beta1_power_right_raw-beta1_power_right_corrected)/(beta1_power_left_raw-beta1_power_left_corrected)+(beta1_power_right_raw-beta1_power_right_corrected)

cCz=np.corrcoef(preCz, postCz)
cCz=cCz[0,1]

print(cCz)

Variable_beta1=[[cCz,beta1_power_left_raw, beta1_power_right_raw, beta1_ratio_raw, beta1_mean_raw, beta1_power_midline_raw, beta1_power_left_corrected, beta1_power_right_corrected, beta1_ratio_corrected, beta1_mean_corrected, beta1_power_midline_corrected, beta1_ratio1, beta1_ratio2]]
#Headers_beta1=["pre-post Fz coef corr","beta1_power_left_raw", "beta1_power_right_raw", "beta1_ratio_raw", "beta1_mean_raw", "beta1_midline_raw", "beta1_power_left_corrected", "beta1_power_right_corrected", "beta1_ratio_corrected", "beta1_mean_corrected","beta1_ratio1", "beta1_ratio2"]
#row_name= subject
DF = pd.DataFrame(Variable_beta1) 
#DF.rename(columns={"pre-post Cz coef corr": "0", "beta1_power_left_raw": "1","beta1_power_right_raw":"2", "beta1_ratio_raw":"3", "beta1_mean_raw":"4", "beta1_midline_raw":"5", "beta1_power_left_corrected":"6", "beta1_power_right_corrected":"7","beta1_ratio_corrected":"8", "beta1_mean_corrected":"9", "beta1_power_midline_corrected":"10", "beta1_ratio1":"11", "beta1_ratio2":"12"})
DF.rename(columns={"pre-post Cz coef corr": "24", "beta1_power_left_raw": "25","beta1_power_right_raw":"26", "beta1_ratio_raw":"27", "beta1_mean_raw":"28", "beta1_midline_raw":"29", "beta1_power_left_corrected":"30", "beta1_power_right_corrected":"31","beta1_ratio_corrected":"32", "beta1_mean_corrected":"33", "beta1_power_midline_corrected":"34", "beta1_ratio1":"35", "beta1_ratio2":"36"})


In [ ]:
# beta2: 
# Define the electrodes of interest
electrodes_OI_right_beta2 = ['F4','C4','P4' ]
electrodes_OI_left_beta2 = ['F3','C3', 'P3']
electrodes_OI_beta2 =['Cz', 'Fz', 'Pz']
freqs = [13, 24]


tf = mne.time_frequency.psd_array_welch(raw.get_data( picks=electrodes_OI_right_beta2), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
beta2_power_right_raw = np.average(np.average(tf[0], axis=0), axis=0)

tf = mne.time_frequency.psd_array_welch(raw.get_data( picks=electrodes_OI_left_beta2), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
beta2_power_left_raw = np.average(np.average(tf[0], axis=0), axis=0)

beta2_ratio_raw =((beta2_power_left_raw-beta2_power_right_raw)/beta2_power_right_raw)*100

beta2_mean_raw=(beta2_power_right_raw+beta2_power_left_raw)/2

tf = mne.time_frequency.psd_array_welch(raw.get_data( picks=electrodes_OI_beta2), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
beta2_power_midline_raw = np.average(np.average(tf[0], axis=0), axis=0)

tf = mne.time_frequency.psd_array_welch(raw_corrected.get_data( picks=electrodes_OI_right_beta2), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
beta2_power_right_corrected = np.average(np.average(tf[0], axis=0), axis=0)

tf = mne.time_frequency.psd_array_welch(raw_corrected.get_data( picks=electrodes_OI_left_beta2), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
beta2_power_left_corrected = np.average(np.average(tf[0], axis=0), axis=0)

beta2_ratio_corrected=((beta2_power_left_corrected-beta2_power_right_corrected)/beta2_power_right_corrected)*100

beta2_mean_corrected=(beta2_power_right_corrected+beta2_power_left_corrected)/2

tf = mne.time_frequency.psd_array_welch(raw_corrected.get_data( picks=electrodes_OI_beta2), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
beta2_power_midline_corrected = np.average(np.average(tf[0], axis=0), axis=0)

beta2_ratio1=alpha_ratio_raw-beta2_ratio_corrected

beta2_ratio2=(beta2_power_left_raw-beta2_power_left_corrected)-(beta2_power_right_raw-beta2_power_right_corrected)/(beta2_power_left_raw-beta2_power_left_corrected)+(beta2_power_right_raw-beta2_power_right_corrected)

cPz=np.corrcoef(prePz, postPz)
cPz=cPz[0,1]

print(cPz)

Variable_beta2=[[cPz,beta2_power_left_raw, beta2_power_right_raw, beta2_ratio_raw, beta2_mean_raw, beta2_power_midline_raw, beta2_power_left_corrected, beta2_power_right_corrected, beta2_ratio_corrected, beta2_mean_corrected, beta2_power_midline_corrected, beta2_ratio1, beta2_ratio2]]
#Headers_beta2=["pre-post Pz coef corr","beta2_power_left_raw", "beta2_power_right_raw", "beta2_ratio_raw", "beta2_mean_raw", "beta2_midline_raw", "beta2_power_left_corrected", "beta2_power_right_corrected", "beta2_ratio_corrected", "beta2_mean_corrected","beta2_ratio1", "beta2_ratio2"]
#row_name= subject
DF = pd.DataFrame(Variable_beta2) 
#DF.rename(columns={"pre-post Pz coef corr": "0", "beta2_power_left_raw": "1","beta2_power_right_raw":"2", "beta2_ratio_raw":"3", "beta2_mean_raw":"4", "beta2_midline_raw":"5", "beta2_power_left_corrected":"6", "beta2_power_right_corrected":"7","beta2_ratio_corrected":"8", "beta2_mean_corrected":"9", "beta2_power_midline_corrected":"10", "beta2_ratio1":"11", "beta2_ratio2":"12"})
DF.rename(columns={"pre-post Pz coef corr": "37", "beta2_power_left_raw": "38","beta2_power_right_raw":"39", "beta2_ratio_raw":"40", "beta2_mean_raw":"41", "beta2_midline_raw":"42", "beta2_power_left_corrected":"43", "beta2_power_right_corrected":"44","beta2_ratio_corrected":"45", "beta2_mean_corrected":"46", "beta2_power_midline_corrected":"47", "beta2_ratio1":"48", "beta2_ratio2":"49"})


In [ ]:
# slow frequency: 
# Define the electrodes of interest
electrodes_OI_right_slowfreq = ['F4','C4','P4' ] ?????
electrodes_OI_left_slowfreq = ['F3','C3', 'P3']
electrodes_OI_slowfreq =['Cz', 'Fz', 'Pz']
freqs = [0.8, 4]


tf = mne.time_frequency.psd_array_welch(raw.get_data( picks=electrodes_OI_right_slowfreq), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
slowfreq_power_right_raw = np.average(np.average(tf[0], axis=0), axis=0)

tf = mne.time_frequency.psd_array_welch(raw.get_data( picks=electrodes_OI_left_slowfreq), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
slowfreq_power_left_raw = np.average(np.average(tf[0], axis=0), axis=0)

slowfreq_ratio_raw =((slowfreq_power_left_raw-slowfreq_power_right_raw)/slowfreq_power_right_raw)*100

slowfreq_mean_raw=(slowfreq_power_right_raw+slowfreq_power_left_raw)/2

tf = mne.time_frequency.psd_array_welch(raw.get_data( picks=electrodes_OI_slowfreq), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
slowfreq_power_midline_raw = np.average(np.average(tf[0], axis=0), axis=0)

tf = mne.time_frequency.psd_array_welch(raw_corrected.get_data( picks=electrodes_OI_right_slowfreq), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
slowfreq_power_right_corrected = np.average(np.average(tf[0], axis=0), axis=0)

tf = mne.time_frequency.psd_array_welch(raw_corrected.get_data( picks=electrodes_OI_left_slowfreq), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
slowfreq_power_left_corrected = np.average(np.average(tf[0], axis=0), axis=0)

slowfreq_ratio_corrected=((slowfreq_power_left_corrected-slowfreq_power_right_corrected)/slowfreq_power_right_corrected)*100

slowfreq_mean_corrected=(slowfreq_power_right_corrected+slowfreq_power_left_corrected)/2

tf = mne.time_frequency.psd_array_welch(raw_corrected.get_data( picks=electrodes_OI_slowfreq), raw.info['sfreq'], fmin=freqs[0], fmax=freqs[1],)
slowfreq_power_midline_corrected = np.average(np.average(tf[0], axis=0), axis=0)

slowfreq_ratio1=alpha_ratio_raw-slowfreq_ratio_corrected

slowfreq_ratio2=(slowfreq_power_left_raw-slowfreq_power_left_corrected)-(slowfreq_power_right_raw-slowfreq_power_right_corrected)/(slowfreq_power_left_raw-slowfreq_power_left_corrected)+(slowfreq_power_right_raw-slowfreq_power_right_corrected)



Variable_slowfreq=[[slowfreq_power_left_raw, slowfreq_power_right_raw, slowfreq_ratio_raw, slowfreq_mean_raw, slowfreq_power_midline_raw, slowfreq_power_left_corrected, slowfreq_power_right_corrected, slowfreq_ratio_corrected, slowfreq_mean_corrected, slowfreq_power_midline_corrected, slowfreq_ratio1, slowfreq_ratio2]]
#Headers_slowfreq=["pre-post Pz coef corr","slowfreq_power_left_raw", "slowfreq_power_right_raw", "slowfreq_ratio_raw", "slowfreq_mean_raw", "slowfreq_midline_raw", "slowfreq_power_left_corrected", "slowfreq_power_right_corrected", "slowfreq_ratio_corrected", "slowfreq_mean_corrected","slowfreq_ratio1", "slowfreq_ratio2"]
#row_name= subject
DF = pd.DataFrame(Variable_slowfreq) 
#DF.rename(columns={"pre-post Pz coef corr": "0", "slowfreq_power_left_raw": "1","slowfreq_power_right_raw":"2", "slowfreq_ratio_raw":"3", "slowfreq_mean_raw":"4", "slowfreq_midline_raw":"5", "slowfreq_power_left_corrected":"6", "slowfreq_power_right_corrected":"7","slowfreq_ratio_corrected":"8", "slowfreq_mean_corrected":"9", "slowfreq_power_midline_corrected":"10", "slowfreq_ratio1":"11", "slowfreq_ratio2":"12"})
DF.rename(columns={"slowfreq_power_left_raw": "50","slowfreq_power_right_raw":"51", "slowfreq_ratio_raw":"52", "slowfreq_mean_raw":"53", "slowfreq_midline_raw":"54", "slowfreq_power_left_corrected":"55", "slowfreq_power_right_corrected":"56","slowfreq_ratio_corrected":"57", "slowfreq_mean_corrected":"58", "slowfreq_power_midline_corrected":"59", "slowfreq_ratio1":"60", "slowfreq_ratio2":"61"})


In [ ]:
print(alpha_power_left_corrected)

In [ ]:
print(alpha_power_right_corrected)

In [ ]:
print(alpha_ratio_corrected)

In [ ]:
print(alpha_ratio1)

In [ ]:
print(alpha_ratio2)

In [ ]:
## Topomaps alpha 
import mne
import numpy as np
import dabest
import matplotlib.pyplot as plt

cmap = 'viridis'

#PRE
psd_raw = raw.compute_psd(fmin=8, fmax=13)
data_raw = np.mean(psd_raw.get_data(), axis=1)
mne.viz.plot_topomap(data_raw, raw.info, cmap=cmap)  
plt.savefig(f'Post_corrections/Reference/CWL_python_block/P{sub:02}_eyes_{modal}-{correction}-topo_alpha-raw.png', dpi=300)

#POST
psd_corrected = raw_corrected.compute_psd(fmin=8, fmax=13)
data_corrected = np.mean(psd_corrected.get_data(), axis=1)
mne.viz.plot_topomap(data_corrected, raw_corrected.info, cmap=cmap)  
plt.savefig(f'Post_corrections/Reference/CWL_python_block/P{sub:02}_eyes_{modal}-{correction}-topo_alpha-corrected-raw.png', dpi=300)

#diff pre-post topo 8-13z
psd_diff=data_raw-data_corrected
mne.viz.plot_topomap(psd_diff, raw.info, cmap=cmap)  
plt.savefig(f'Post_corrections/Reference/CWL_python_block/P{sub:02}_eyes_{modal}-{correction}-topo_alpha-DIFF_rawcorrected.png', dpi=300)

#diff post-pre topo 8-13z
psd_diff=data_corrected-data_raw
mne.viz.plot_topomap(psd_diff, raw.info, cmap=cmap)  
plt.savefig(f'Post_corrections/Reference/CWL_python_block/P{sub:02}_eyes_{modal}-{correction}-topo_alpha-DIFF_correctedraw.png', dpi=300)

In [ ]:
## Topomaps all frequencies >> 1-30Hz
#PRE RAW
psd_raw = raw.compute_psd(fmin=1, fmax=30)
data_raw = np.mean(psd_raw.get_data(), axis=1)
mne.viz.plot_topomap(data_raw, raw.info, cmap=cmap)  
plt.savefig(f'Post_corrections/Reference/CWL_python_block/P{sub:02}_eyes_{modal}-{correction}-topo_all_freq-raw.png', dpi=300)

#POST CORRECTED
psd_corrected = raw_corrected.compute_psd(fmin=1, fmax=30)
data_corrected = np.mean(psd_corrected.get_data(), axis=1)
mne.viz.plot_topomap(data_corrected, raw_corrected.info, cmap=cmap)  
plt.savefig(f'Post_corrections/Reference/CWL_python_block/P{sub:02}_eyes_{modal}-{correction}-topo_all_freq-corrected-raw.png', dpi=300)

#PRE-POST
psd_diff=data_raw-data_corrected
mne.viz.plot_topomap(psd_diff, raw.info, cmap=cmap)  
plt.savefig(f'Post_corrections/Reference/CWL_python_block/P{sub:02}_eyes_{modal}-{correction}-topo_all_freq-DIFF_rawcorrected.png', dpi=300)

#POST-PRE
psd_diff=data_corrected-data_raw
mne.viz.plot_topomap(psd_diff, raw.info, cmap=cmap)  
plt.savefig(f'Post_corrections/Reference/CWL_python_block/P{sub:02}_eyes_{modal}-{correction}-topo_all_freq-DIFF_correctedraw.png', dpi=300)

In [ ]:
print(GA_PA_CWL_RecView.info)